In [ ]:
from google.colab import userdata
userdata.get('API_TOKEN')

'r8_JdJprP5Sb1PEfoq1GwwBE3GOkjKRk2k25jJ4A'

In [ ]:
# Set API Token
os.environ["REPLICATE_API_TOKEN"] = "r8_JdJprP5Sb1PEfoq1GwwBE3GOkjKRk2k25jJ4A"
client = replicate.Client(api_token=os.environ["REPLICATE_API_TOKEN"])

# Load dataset
try:
    df = pd.read_csv("amazon.csv")
    df = df[['product_name', 'rating', 'actual_price', 'review_content']].dropna().head(50)
    print("✅ Dataset berhasil dimuat:", df.shape)
except Exception as e:
    print("❌ ERROR:", e)

✅ Dataset berhasil dimuat: (50, 4)


In [ ]:
# === 1. INSTALL & SETUP ===
!pip install replicate --quiet

import os
import replicate
import pandas as pd
from tqdm.auto import tqdm
import time
tqdm.pandas()

# === 2. SET API TOKEN ===
os.environ["REPLICATE_API_TOKEN"] = "r8_JdJprP5Sb1PEfoq1GwwBE3GOkjKRk2k25jJ4A"
client = replicate.Client(api_token=os.environ["REPLICATE_API_TOKEN"])

# === 3. LOAD DATASET ===
try:
    df = pd.read_csv("amazon.csv")
    df = df[['product_name', 'rating', 'actual_price', 'review_content']].dropna().head(15)
    print("✅ Data berhasil dimuat:")
    print(df[['product_name', 'rating']])
except Exception as e:
    print(f"❌ ERROR loading dataset: {e}")

✅ Data berhasil dimuat:
                                         product_name rating
0   Wayona Nylon Braided USB to Lightning Fast Cha...    4.2
1   Ambrane Unbreakable 60W / 3A Fast Charging 1.5...    4.0
2   Sounce Fast Phone Charging Cable & Data Sync U...    3.9
3   boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...    4.2
4   Portronics Konnect L 1.2M Fast Charging 3A 8 P...    4.2
5   pTron Solero TB301 3A Type-C Data and Fast Cha...    3.9
6   boAt Micro USB 55 Tangle-free, Sturdy Micro US...    4.1
7              MI Usb Type-C Cable Smartphone (Black)    4.3
8   TP-Link USB WiFi Adapter for PC(TL-WN725N), N1...    4.2
9   Ambrane Unbreakable 60W / 3A Fast Charging 1.5...    4.0
10  Portronics Konnect L POR-1081 Fast Charging 3A...    4.3
11  boAt Rugged v3 Extra Tough Unbreakable Braided...    4.2
12  AmazonBasics Flexible Premium HDMI Cable (Blac...    4.4
13  Portronics Konnect CL 20W POR-1067 Type-C to 8...    4.2
14  Portronics Konnect L 1.2M POR-1401 Fast Chargi...    4.1


In [ ]:
for event in client.stream(
    "ibm-granite/granite-3.3-8b-instruct",
    input={
        "prompt": "Apa itu artificial intelligence dalam 2 kalimat?",
        "temperature": 0.6,
        "max_new_tokens": 100
    }
):
    print(str(event), end="")


Artificial Intelligence (AI) adalah bidang ilmu yang mengacu pada pembuatan sistem komputer atau mesin yang mampu melakukan tugas-tugas yang biasanya memerlukan inteligensi manusia, seperti belajar, penyelesaian masalah, dan pengakuan patron. AI meliputi berbagai sub-disiplin, termasuk pemrograman, logika, ilmu komputer, dan matematika.

# LLM DENGAN MENGGUNAKAN AI MODEL

In [ ]:
# CLASSIFICATION
def classify_product(row):
    prompt = f"""Apakah produk berikut layak dibeli? Jawab hanya dengan 'Worth to Buy' atau 'Not Worth to Buy'.

Nama Produk: {row['product_name']}
Rating: {row['rating']}
Harga: {row['actual_price']}

Jawaban:"""
    try:
        output = replicate.run(
            "ibm-granite/granite-3.3-8b-instruct",
            input={"prompt": prompt, "temperature": 0.4, "max_new_tokens": 30}
        )
        result = "".join(output)
        return "Worth to Buy" if "worth" in result.lower() else "Not Worth to Buy"
    except Exception as e:
        return f"[ERROR Classification]: {e}"

# SUMMARIZATION
def summarize_review(row):
    prompt = f"""Buat ringkasan ulasan produk ini dalam 3–5 kalimat berdasarkan review berikut:

Review:
"{row['review_content']}"
"""
    try:
        output = replicate.run(
            "ibm-granite/granite-3.3-8b-instruct",
            input={"prompt": prompt, "temperature": 0.6, "max_new_tokens": 300}
        )
        return "".join(output).strip()
    except Exception as e:
        return f"[ERROR Summary]: {e}"

# SENTIMENT ANALYSIS
def sentiment_analysis(row):
    prompt = f"""Tentukan sentimen dari ulasan berikut. Jawab hanya dengan satu kata: Positive, Neutral, atau Negative.

Ulasan:
"{row['review_content']}"

Sentimen:"""
    try:
        output = replicate.run(
            "ibm-granite/granite-3.3-8b-instruct",
            input={"prompt": prompt, "temperature": 0.4, "max_new_tokens": 10}
        )
        return "".join(output).strip()
    except Exception as e:
        return f"[ERROR Sentiment]: {e}"

results = []

for idx, row in df.iterrows():
    classification = classify_product(row)
    sentiment = sentiment_analysis(row)
    summary = summarize_review(row)

    output_text = f"""
📦 Produk #{idx+1}: {row['product_name']}
⭐ Rating: {row['rating']}
💰 Harga: {row['actual_price']}

🧠 Klasifikasi: Produk ini dianggap "{classification}" oleh LLM.
💬 Sentimen ulasan dinilai sebagai: {sentiment}.
📝 Ringkasan Ulasan:
{summary}

──────────────────────────────
"""
    print(output_text)
    results.append(output_text)

# Opsional: Simpan ke file .txt
with open("hasil_5_produk_amazon.txt", "w") as f:
    f.writelines(results)


📦 Produk #1: Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)
⭐ Rating: 4.2
💰 Harga: ₹1,099

🧠 Klasifikasi: Produk ini dianggap "Worth to Buy" oleh LLM.
💬 Sentimen ulasan dinilai sebagai: Positive.
📝 Ringkasan Ulasan:
This product is praised for its durability and reliable charging, described as fast. Users express satisfaction with its quality and recommend it as a good value for money. Despite one user noting the charging speed is slightly slower than the original iPhone cable, overall feedback is positive, with no reported issues and a commendable replacement service. The cable's sturdiness is also highlighted.

──────────────────────────────


📦 Produk #2: Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C Cable for Smartphones, Tablets, Laptops & other Type C devices, PD Technology, 480Mbps Data Sync, Quick Charge 3.0 (RCT15A, Black)
⭐ Rating: 4.0
💰 Harga:

# ANALISIS MENGGUNAKAN AI

In [ ]:
# Load Dataset ===
import pandas as pd

try:
    df = pd.read_csv("amazon.csv")
    print("✅ Dataset 'amazon.csv' berhasil dimuat.")
    print("📌 Kolom tersedia:", df.columns.tolist())

    sample_df = df[['product_name', 'rating', 'actual_price', 'review_content']].dropna().head(100)
    print(f"✅ Sample data yang akan diproses: {len(sample_df)} produk")

except FileNotFoundError:
    print("❌ ERROR: File 'amazon.csv' tidak ditemukan.")
    sample_df = None

# Setup Replicate ===
import replicate
from google.colab import userdata
import time
from tqdm.auto import tqdm
tqdm.pandas()

try:
    REPLICATE_TOKEN = userdata.get('API_TOKEN')
    client = replicate.Client(api_token=REPLICATE_TOKEN)
    print("✅ Replicate client initialized.")
except Exception as e:
    print(f"❌ ERROR saat inisialisasi Replicate: {e}")
    client = None

# Retry Handler ===
def replicate_retry(run_fn, max_retries=5, delay=5):
    for attempt in range(max_retries):
        try:
            return run_fn()
        except Exception as e:
            print(f"⏳ Retry {attempt+1}/{max_retries} after error: {e}")
            time.sleep(delay)
    return "[ERROR] Max retries exceeded"

# CLASSIFICATION ===
def classify_product(row):
    if client is None:
        return "[ERROR] Replicate client not initialized"
    prompt = f"""Apakah produk berikut layak dibeli? Jawab hanya dengan 'Worth to Buy' atau 'Not Worth to Buy'. Jangan tambahkan penjelasan.

Produk:
- Nama Produk: {row['product_name']}
- Rating: {row['rating']}
- Harga: {row['actual_price']}

Jawaban:"""
    result = replicate_retry(lambda: ''.join(list(client.run(
        "ibm-granite/granite-3.3-8b-instruct",
        input={"prompt": prompt, "temperature": 0.5, "max_new_tokens": 30}
    ))).strip())
    # Post-process agar hanya dua nilai:
    if "not" in result.lower():
        return "Not Worth to Buy"
    elif "worth" in result.lower():
        return "Worth to Buy"
    else:
        return "[INVALID OUTPUT]"

# SUMMARIZATION ===
def summarize_product(row, classification_result):
    if client is None:
        return "[ERROR] Replicate client not initialized"
    prompt = f"""Buat ringkasan penilaian produk berikut dalam 3–5 kalimat berdasarkan rating dan harga. Jelaskan mengapa produk ini dianggap '{classification_result}'.

- Nama Produk: {row['product_name']}
- Rating: {row['rating']}
- Harga: {row['actual_price']}
"""
    return replicate_retry(lambda: ''.join(list(client.run(
        "ibm-granite/granite-3.3-8b-instruct",
        input={"prompt": prompt, "temperature": 0.6, "max_new_tokens": 300}
    ))).strip())

# SENTIMENT ANALYSIS
def sentiment_analysis(row):
    prompt = f"""Tentukan sentimen dari ulasan berikut. Jawab hanya dengan satu kata: Positive, Neutral, atau Negative.

Ulasan:
"{row['review_content']}"

Sentimen:"""
    try:
        output = replicate_retry(lambda: ''.join(list(client.run(
            "ibm-granite/granite-3.3-8b-instruct",
            input={"prompt": prompt, "temperature": 0.4, "max_new_tokens": 10}
        ))).strip())
        return output
    except Exception as e:
        return f"[ERROR Sentiment]: {e}"

# PROSES
if sample_df is not None and client is not None:
    print("🚀 Memulai proses klasifikasi, ringkasan, dan analisis sentimen dengan LLM...")

    # 1️⃣ CLASSIFICATION
    sample_df['Classification'] = sample_df.progress_apply(classify_product, axis=1)

    # 2️⃣ SUMMARIZATION
    time.sleep(3)
    sample_df['Summary'] = sample_df.progress_apply(
        lambda row: summarize_product(row, row['Classification']), axis=1
    )

    # 3️⃣ SENTIMENT ANALYSIS
    time.sleep(3)
    sample_df['Sentiment'] = sample_df.progress_apply(sentiment_analysis, axis=1)

# Tampilkan dan Simpan
    from IPython.display import display
    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.max_rows', 100)
    display(sample_df[['product_name', 'rating', 'actual_price', 'Classification', 'Summary', 'Sentiment']])

    # Simpan ke file CSV
    sample_df[['product_name', 'rating', 'actual_price', 'Classification', 'Summary', 'Sentiment']].to_csv(
        "hasil_llm_100_amazon.csv", index=False
    )
    print("✅ File hasil disimpan sebagai 'hasil_llm_100_amazon.csv'.")

else:
    print("⚠️ Proses dihentikan karena sample_df atau token belum tersedia.")

✅ Dataset 'amazon.csv' berhasil dimuat.
📌 Kolom tersedia: ['product_id', 'product_name', 'category', 'discounted_price', 'actual_price', 'discount_percentage', 'rating', 'rating_count', 'about_product', 'user_id', 'user_name', 'review_id', 'review_title', 'review_content', 'img_link', 'product_link']
✅ Sample data yang akan diproses: 100 produk
✅ Replicate client initialized.
🚀 Memulai proses klasifikasi, ringkasan, dan analisis sentimen dengan LLM...


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

,product_name,rating,actual_price,Classification,Summary,Sentiment
0,"Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)",4.2,"₹1,099",Not Worth to Buy,"The Wayona Nylon Braided USB to Lightning Cable, despite its decent 4.2-star rating, is not recommended as a worthwhile purchase at ₹1,099. While the nylon braided design offers durability and the cable supports fast charging and data sync for various Apple devices, the high price point seems disproportionate compared to similar, often cheaper alternatives available in the market. Consumers can find comparable quality cables at lower costs, making this option less appealing value-wise. Additionally, some user reviews mention compatibility issues with certain devices or slower charging speeds than expected, further questioning its overall value proposition.",Positive
1,"Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C Cable for Smartphones, Tablets, Laptops & other Type C devices, PD Technology, 480Mbps Data Sync, Quick Charge 3.0 (RCT15A, Black)",4.0,₹349,Worth to Buy,"The Ambrane Unbreakable 60W Type-C cable earns a 'Worth to Buy' rating, scoring 4.0 out of 5, making it a reliable choice for charging smartphones, tablets, laptops, and other Type-C devices. Priced at ₹349, it offers fast charging with PD Technology and Quick Charge 3.0, ensuring efficient power delivery. Its 1.5m braided construction promises durability and tangle-free convenience. The 480Mbps data sync capability further adds to its value, making it a feature-rich, cost-effective solution for your charging needs.","Negative (due to initial quality issue with fungus inside the connector, despite resolving it with cleaning, and mention of potential distributor storage conditions causing harm)"
2,"Sounce Fast Phone Charging Cable & Data Sync USB Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini & iOS Devices",3.9,"₹1,899",Not Worth to Buy,"Produk ini, Sounce Fast Phone Charging Cable & Data Sync USB Cable, terkait dengan kompatibiliti yang luas untuk berbagai model iPhone dan iPad, tetapi harganya yang tinggi (₹1,899) dan rating hanya 3.9 sangat menyebabkan keberatan untuk membelinya. Meskipun rating tersebut masih menunjukkan kualitas yang cukup baik, harga yang tinggi tidak diwakili oleh kemampuan yang signifikan lebih baik dari cable standar atau alternatif yang lebih murah. Pelanggan melaporkan kecepatan penyimpanan dan peningkatan koneksi yang cukup tetapi tidak menunjukkan perbedaan signifikan untuk membuat keputusan membelinya, terutama dengan harga yang tinggi. Oleh karena itu, produk ini dianggap tidak layak untuk dibeli, karena harga yang tinggi tidak dijustifikasi oleh manfaat yang ditawarkan.","Negative (due to ""not quite durable and sturdy"")"
3,"boAt Deuce USB 300 2 in 1 Type-C & Micro USB Stress Resistant, Tangle-Free, Sturdy Cable with 3A Fast Charging & 480mbps Data Transmission, 10000+ Bends Lifespan and Extended 1.5m Length(Martian Red)",4.2,₹699,Worth to Buy,"The boAt Deuce USB 300 2-in-1 Type-C & Micro USB cable is a worthwhile purchase, garnering a solid 4.2-star rating from users. Priced at ₹699, it offers durability with a stress-resistant design and tangle-free construction, promising a lifespan of over 10000 bends. The cable supports fast charging up to 3A and data transmission up to 480mbps, along with an extended 1.5m length in a vibrant Martian Red color. Its compatibility with both Type-C and Micro USB ports, coupled with its robust features and reasonable price, makes it an excellent 'Worth to Buy' option for those in search of a reliable, high-performance cable.",Negative
4,"Portronics Konnect L 1.2M Fast Charging 3A 8 Pin USB Cable with Charge & Sync Function for iPhone, iPad (Grey)",4.2,₹399,Worth to Buy,"The Portronics Konnect L 1.2M Fast Charging 3A 8 Pin USB Cable, rated 4.2 out of 5, is a worthwhile purchase for its efficient 

✅ File hasil disimpan sebagai 'hasil_llm_100_amazon.csv'.
